### Setup & Imports

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip ffmpeg libvulkan1
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=$(curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE)
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver

pip install openai-whisper selenium chromedriver-autoinstaller mistralai yfinance fastapi[all] pyngrok uvicorn langchain-community langchain_mistralai chromadb pypdf langchain nest_asyncio faiss-cpu tiktoken qwen-vl-utils gptcache langgraph langchain_experimental -q
pip install git+https://github.com/huggingface/transformers datasets[audio] accelerate -q

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,391 kB]
Get:14 https://r2u.stat.illinois.ed

In [ ]:
import os
import re
import io
import sys
import time
import uuid
import logging
import warnings

import asyncio
from contextlib import asynccontextmanager
from typing import Dict, List, Union

import numpy as np
import pandas as pd
import torch

from fastapi import FastAPI, HTTPException, UploadFile, File, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

from langchain.agents import Tool, AgentExecutor
from langchain_experimental.agents import create_csv_agent
from langchain.document_loaders import PyPDFLoader
from langchain.globals import set_llm_cache
from langchain_mistralai import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import ToolException
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

import chromadb
import whisper
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions

from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

from pyngrok import ngrok
import yfinance as yf
from googlesearch import search

from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from langchain_community.cache import GPTCache

import urllib.request
import nest_asyncio
import uvicorn

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
warnings.filterwarnings('ignore')
logger = logging.getLogger(__name__)
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
#os.environ["MISTRAL_API_KEY"] = ""
#NGROK_AUTH_TOKEN = ""

### Modelling

In [ ]:
class MistralEmbeddingFunction(EmbeddingFunction[Documents]):
    def __init__(self, api_key: str, model_name: str) -> None:
        from mistralai import Mistral
        self._client = Mistral(api_key=api_key)
        self._model_name = model_name

    def __call__(self, input: Union[Documents, str]) -> Embeddings:
        texts = input if isinstance(input, list) else [input]
        return [embedding_obj.embedding for embedding_obj in self._client.embeddings.create(model=self._model_name, inputs=texts).data]

In [ ]:
embedding_model = MistralAIEmbeddings(model="mistral-embed")
mistral_ef = MistralEmbeddingFunction(api_key=os.environ["MISTRAL_API_KEY"], model_name="mistral-embed")

chroma_client = chromadb.PersistentClient(path="./db")
llm = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0.7,
    max_tokens=10000
)

memory = MemorySaver()
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

qwen_processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
qwen_model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto")
voice_model = whisper.load_model("large-v3-turbo",device="cuda")

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

100%|██████████████████████████████████████| 1.51G/1.51G [00:14<00:00, 110MiB/s]


In [ ]:
class Query(BaseModel):
    text: str

class FinancialReport(BaseModel):
    balance_sheet: Dict
    pnl_statement: Dict
    stock_price_trend: List[float]

class Portfolio(BaseModel):
    stocks: List[str]
    bonds: List[str]
    mutual_funds: List[str]
    allocation: Dict[str, float]

class FinancialAdvice(BaseModel):
    recommendations: List[str]
    risk_assessment: str
    action_items: List[str]

class PortfolioPlannerInput(BaseModel):
    portfolio_pdf: UploadFile
    risk_tolerance: float
    min_profit_margin: float
    maturity_time: int

In [ ]:
# PDF Processing
class PDFProcessor:
    def __init__(self):
        self.text_splitter = CharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separator="\n"
        )

    async def process_pdf(self, file: UploadFile) -> Chroma:
        temp_path = f"temp_{file.filename}"
        with open(temp_path, "wb") as temp_file:
            content = await file.read()
            temp_file.write(content)

        try:
            loader = PyPDFLoader(temp_path)
            documents = loader.load()
            texts = self.text_splitter.split_documents(documents)

            vectorstore = Chroma.from_documents(
                documents=texts,
                embedding=embedding_model,
                client=chroma_client,
                collection_name=file.filename
            )

            return vectorstore
        finally:
            os.remove(temp_path)

### Tools & Utility Functions

In [ ]:
# def get_hashed_name(name):
#     return hashlib.sha256(name.encode()).hexdigest()

# def init_gptcache(cache_obj: Cache, llm: str):
#     hashed_llm = get_hashed_name(llm)
#     init_similar_cache(cache_obj=cache_obj, data_dir=f"similar_cache_{hashed_llm}")

# set_llm_cache(GPTCache(init_gptcache))

@tool("market_analysis")
async def analyze_market(companies: List[str]) -> str:
    """Analyzes market trends for given companies."""
    try:
        scraped_data = await scrape_company_data(companies)
        analysis = await analyze_with_llm(scraped_data)
        return analysis
    except Exception as e:
        raise ToolException(f"Error in market analysis: {str(e)}")

@tool("portfolio_planning")
async def plan_portfolio(input_data: str) -> str:
    """Plans investment portfolio based on given criteria."""
    try:
        input_obj = PortfolioPlannerInput.parse_raw(input_data)
        vectorstore = await pdf_processor.process_pdf(input_obj.portfolio_pdf)
        recommendations = await generate_portfolio_recommendations(
            vectorstore,
            input_obj.risk_tolerance,
            input_obj.min_profit_margin,
            input_obj.maturity_time
        )
        return str(Portfolio(**recommendations))
    except Exception as e:
        raise ToolException(f"Error in portfolio planning: {str(e)}")

@tool("financial_advice")
async def get_financial_advice(document_content: str) -> str:
    """Provides financial advice based on given document content."""
    try:
        vectorstore = await pdf_processor.process_text(document_content)
        advice = await generate_financial_advice(vectorstore)
        return str(FinancialAdvice(**advice))
    except Exception as e:
        raise ToolException(f"Error in providing financial advice: {str(e)}")

KB = Chroma(client=chroma_client,collection_name="Knowledge_Base",embedding_function=mistral_ef)
retriever = KB.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_Knowledge_Base_information",
    "Search and return information about Stocks from Knowledge Base.",
)

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup logic
    try:
        # Configure ngrok
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)

        # Start ngrok tunnel
        public_url = ngrok.connect(8000).public_url
        print(f"ngrok tunnel created: {public_url}")
        app.state.public_url = public_url

        yield  # This is where the application runs
    finally:
        # Shutdown logic
        ngrok.kill()

In [ ]:
async def scrape_company_data(companies: List[str]):
    # Placeholder implementation for company data scraping
    company_data = {}
    for company in companies:
        company_data[company] = {
            "stock_price": 100 + np.random.randn() * 10,
            "market_cap": 1e9 + np.random.randn() * 1e8,
            "pe_ratio": 15 + np.random.randn() * 5
        }
    return company_data

async def analyze_with_llm(data: Dict):
    prompt = f"Analyze the following company data and provide insights: {data}"
    response = await llm.ainvoke(prompt)
    return response.content

async def generate_portfolio_recommendations(vectorstore, risk_tolerance, min_profit_margin, maturity_time):
    # Placeholder implementation
    return {
        "stocks": ["AAPL", "GOOGL", "MSFT"],
        "bonds": ["US Treasury 10Y", "Corporate Bond XYZ"],
        "mutual_funds": ["Vanguard 500 Index Fund", "Fidelity Contrafund"],
        "allocation": {"stocks": 0.6, "bonds": 0.3, "mutual_funds": 0.1}
    }

async def generate_financial_advice(vectorstore):
    # Placeholder implementation
    return {
        "recommendations": ["Diversify your portfolio", "Increase emergency savings"],
        "risk_assessment": "Moderate risk tolerance",
        "action_items": ["Review insurance coverage", "Set up automatic investments"]
    }

async def process_vqa_with_qwen(img_path, question):
    # Create message structure
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img_path},
                {"type": "text", "text": question},
            ],
        }
    ]

    # Prepare inputs for inference
    text = qwen_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = qwen_processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # Generate output
    generated_ids = qwen_model.generate(**inputs, max_new_tokens=1024)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    # Decode output
    output_text = qwen_processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text

### Backend Run

In [ ]:
app = FastAPI(lifespan=lifespan)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

pdf_processor = PDFProcessor()

@app.get("/")
async def start():
    return {"message": "Server is up and running"}

# API endpoints
@app.post("/upload-pdf")
async def upload_pdf(file: UploadFile = File(...)):
    if not file.filename.endswith('.pdf'):
        raise HTTPException(status_code=400, detail="Only PDF files are allowed")

    try:
        vectorstore = await pdf_processor.process_pdf(file)
        return {"message": f"Successfully processed {file.filename}"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/query-chroma")
async def query_chroma(query: str, collection_name: str):
    try:
        collection = chroma_client.get_collection(collection_name, embedding_function=mistral_ef)
        context = collection.query(
            query_texts=[query],  # Query provided by the user
            n_results=5  # Return top 5 most relevant documents
        )['documents']
        context = '\n'.join(item for sublist in context for item in sublist)
        full_query = f"{query} {context}"  # Append context to the query
        response = await llm.ainvoke(full_query)

        return {"llm_response": response.content}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/csv-query")
async def query_csv(question: str = Form(...), file: UploadFile = File(...)):
    if not file.filename.endswith('.csv'):
        raise HTTPException(status_code=400, detail="Only CSV files are allowed")
    try:
      csv_content = await file.read()
      df = pd.read_csv(io.BytesIO(csv_content))
      file_path = f"{uuid.uuid4()}.csv"
      df.to_csv(file_path, index=False)
      agent = create_csv_agent(llm, file_path, allow_dangerous_code=True)
      try:
        response = agent.run(question)
      except Exception as e:
        matches = re.search(r'Final Answer:\s*(.*)', str(e))
        if matches:
          response = matches.group(1)
      return {"response": response}
    except Exception as e:
      raise HTTPException(status_code=500, detail=str(e))

@app.post("/crawl-news")
async def crawl_news(ticker: str):
    return {"news": yf.Ticker(ticker).news}

@app.post("/table-chart-qa")
async def table_qa(question: str = Form(...), image: UploadFile = File(...)):
    img_path = f"{uuid.uuid4()}.jpg"
    with open(img_path, "wb") as img_file:
        img_file.write(image.file.read())
    response = await process_vqa_with_qwen(img_path, question)
    return {"answer": response}

@app.post("/query")
async def process_query(query: Query):
    tools = [YahooFinanceNewsTool(), retriever_tool, analyze_market, plan_portfolio, get_financial_advice]
    system_prompt = "You are a Financial Assistant called Anzen."
    config = {"configurable": {"thread_id": "thread-1"}}
    inputs = {"messages": [("user", query.text)]}
    memory = MemorySaver()
    agent = create_react_agent(llm, tools=tools, state_modifier=system_prompt, checkpointer=memory)
    response = await agent.ainvoke(inputs, config=config)
    return {"response": response['messages'][-1].content}

@app.post('/web-search')
async def web_search_endpoint(query_req: Query):
    query = query_req.text
    # YT part
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(f"https://www.youtube.com/results?search_query={query}")
    wait = WebDriverWait(driver, 2)
    user_data = driver.find_elements(By.XPATH, '//a[@id="video-title"]')
    linksYT = []
    for i in user_data:
        attr = i.get_attribute("href")
        if 'shorts' not in i.get_attribute("href"):
            linksYT.append(attr[attr.find("=") + 1:attr.find("&")])

    return {"youtube" : linksYT}

@app.post("/LLM")
async def process_query(query: Query):
    response = await llm.ainvoke(query.text)
    return {"response": response.content}

@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    try:
      audio_path = f"{uuid.uuid4()}.webm"
      with open(audio_path, "wb") as f:
          f.write(await file.read())
      result = voice_model.transcribe(whisper.pad_or_trim(whisper.load_audio(audio_path)))
      text = result["text"]
      src_lang = result["language"]
      os.remove(audio_path)
      return {"text": text,"src_lang":src_lang}
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/financial-reports/{company}")
async def get_financial_reports(company: str) -> FinancialReport:
    # Placeholder implementation
    return FinancialReport(
        balance_sheet={"assets": 1000000, "liabilities": 500000},
        pnl_statement={"revenue": 2000000, "expenses": 1500000},
        stock_price_trend=[100, 102, 98, 105, 110]
    )

@app.post("/ranked-stocks")
async def get_ranked_stocks(lst: List[str]):
    # Placeholder implementation
    return {"ranked_stocks": lst}

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [292]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ngrok tunnel created: https://8810-34-168-194-153.ngrok-free.app
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "OPTIONS /crawl-news?ticker=NVDA HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "POST /crawl-news?ticker=NVDA HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "OPTIONS /web-search HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "POST /web-search HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "POST /crawl-news?ticker=NVDA HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "POST /web-search HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "OPTIONS /LLM HTTP/1.1" 200 OK
INFO:     2401:4900:8814:2ee4:3d97:261d:77dc:dad1:0 - "POST /LLM HTTP/1.1" 200 OK
